In [1]:
import torch
import torch.nn as nn
import numpy as np
from sf_examples.nethack.models.scaled import CharColorEncoderResnet, ScaledNet
from sf_examples.nethack.models.utils import model_layernorm, inject_layernorm_before_activation, linear_layernorm, sequential_layernorm, replace_batchnorm_with_layernorm

# old

In [2]:
# def replace_batchnorm_with_layernorm(module):
#     for name, child in module.named_children():
#         if isinstance(child, nn.BatchNorm2d):
#             # Replace BatchNorm2d with LayerNorm
#             num_features = child.num_features
#             new_layer = nn.LayerNorm([num_features])
#             setattr(module, name, new_layer)
#         else:
#             replace_batchnorm_with_layernorm(child)

# def inject_layernorm_before_activation(module):   
#     for name, child in module.named_children():
#         if isinstance(child, nn.Sequential):
#             new_children = []
#             for i, (sub_name, sub_child) in enumerate(child.named_children()):
#                 # Check if the next layer is an activation function and the current layer is not LayerNorm
#                 new_children.append(sub_child)
#                 if i+1 < len(child) and isinstance(child[i+1], (nn.ELU, nn.ReLU, nn.Tanh)) and not isinstance(child[i], nn.LayerNorm):
#                     # Inject LayerNorm before activation function
#                     if hasattr(sub_child, "out_features"):
#                         features = [sub_child.out_features]
#                     elif hasattr(sub_child, "out_channels"):
#                         features = [sub_child.out_channels]
#                     new_layer = nn.LayerNorm(features)
#                     new_children.append(new_layer)
#             setattr(module, name, nn.Sequential(*new_children))
#         else:
#             inject_layernorm_before_activation(child)

# def remove_batchnorm_layers(module):
#     for name, child in list(module.named_children()):
#         if isinstance(child, nn.BatchNorm2d):
#             # Remove the BatchNorm2d layer by deleting it
#             delattr(module, name)
#         elif isinstance(child, nn.Sequential):
#             # Rebuild the Sequential container without BatchNorm2d layers
#             new_children = []
#             for sub_child in child.children():
#                 if not isinstance(sub_child, nn.BatchNorm2d):
#                     new_children.append(sub_child)
#             # Recursively apply to nested Sequential containers
#             new_sequential = nn.Sequential(*new_children)
#             remove_batchnorm_layers(new_sequential)
#             setattr(module, name, new_sequential)
#         else:
#             remove_batchnorm_layers(child)

# def modify_model(model):
#     replace_batchnorm_with_layernorm(model)
#     inject_layernorm_before_activation(model)

# new

In [2]:
from sample_factory.cfg.arguments import load_from_checkpoint
from sample_factory.utils.attr_dict import AttrDict
from sample_factory.algo.utils.env_info import extract_env_info
from sample_factory.algo.utils.make_env import make_env_func_batched

from sf_examples.nethack.train_nethack import parse_nethack_args, register_nethack_components

/home/bartek/Workspace/ideas/sample-factory/sf_examples/nethack/render_utils/Hack-Regular.ttf


In [3]:
env_name = "challenge"
register_nethack_components()
cfg = parse_nethack_args(
    [
        f"--env={env_name}",
    ]
)

[2024-05-23 07:55:09,619][121495] register_encoder_factory: <function make_nethack_encoder at 0x70cd648dbd00>
[2024-05-23 07:55:09,620][121495] register_actor_critic_factory: <function make_nethack_actor_critic at 0x70cd648dbeb0>


In [4]:
env = make_env_func_batched(cfg, env_config=AttrDict(worker_index=0, vector_index=0, env_id=0))
env_info = extract_env_info(env, cfg)

obs_space = env_info.obs_space

In [5]:
model = ScaledNet(cfg, obs_space=obs_space)

In [13]:
# Function to register hooks
handles = []

def register_hooks(model):
    def hook(module, input, output):
        module.output_shape = output.shape
        # print(f"{module.__class__.__name__} output shape: {output.shape}")

    for name, child in model.named_children():
        if isinstance(child, (nn.Linear, nn.Conv1d, nn.Conv2d)):
            handle = child.register_forward_hook(hook)
            handles.append(handle)
        else:
            register_hooks(child)

model = ScaledNet(cfg, obs_space=obs_space)

register_hooks(model)

obs, info = env.reset()

model(obs)

tensor([[0.0000, 0.0812, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)

In [14]:
replace_batchnorm_with_layernorm(model)
inject_layernorm_before_activation(model)

In [16]:
model(obs)

tensor([[0.0000, 0.6838, 0.0000,  ..., 0.0000, 0.3041, 0.3730]],
       grad_fn=<ReluBackward0>)